In [1]:
# Imports

import numpy as np
import pandas as pd

from random import shuffle

from func_matching import calculate_match
from func_args import get_args

from func_aquisition import adquisition_survey
from func_outputs import outputs, create_matrix_matching
from func_normalize import *

def least_common(queue):
    ''' Ordenar por menos número de entrevistas en la cola '''
    res = sorted(queue, key=queue.get, reverse=False)
    return res

def get_next_interview(lista_matching_ordenada, used, students_dict_queue): 
    
    # ordena los estudiantes por prioridad según el que tenga menos entrevistas concertadas
    students_ordered = least_common(students_dict_queue) 

    for student in students_ordered: # según el orden de los menos entrevistados
        if student not in used: 
            for match in lista_matching_ordenada:
                if match['company'] not in used and student == match['student']: # TO DO all() 
                    return match # Devuelve el primer match válido, según el orden de least_common()
    return None # Ningún matcha válido
    
def get_rondas(lista_matching, n_rondas, students, companies): 
    # lista_matching_ordenada = sorted(lista_matching, key = lambda x: x['weight'], reverse=True)
    lista_matching_ordenada = sorted(lista_matching, key = lambda x: x['weight'], reverse=True)
    students_dict_queue = generate_student_interviews(students)
    
    res = []
    for ronda in range(n_rondas): # rondas
        used = []
        for _ in range(len(companies)): # num mesas = num companies
            
            m = get_next_interview(lista_matching_ordenada, used, students_dict_queue)
            if m == None: 
                break
            student, company, w = m['student'], m['company'], m['weight']
            used.extend([student, company]) # añadimos empresa y compañia para que no vuelvan a aparecer en esta ronda
            lista_matching_ordenada.remove(m) # eliminamos el matching para que no se repita

            res.append({'ronda': ronda, 'company': company, 'student': student, 'weight': w})
            students_dict_queue[student] += 1
    return res    

def generate_student_interviews(students_list): 
    '''
    input: list of students
    output: dictionary key (student), value (0)
    '''
    return {student: 0 for student in students_list}


def main(): 
    # Captura de argumentos. 
    bootcamp, companies_filename, students_filename, n_rondas = get_args()
    
    # DFs limpios
    df_companies, df_students = adquisition_survey(bootcamp, companies_filename, students_filename)
    
    # Normalize
    students, companies = normalize_2dfs(df_students, df_companies)

    # TO DO: crear la tabla matching
    lista_matching = calculate_match(students, companies, bootcamp)
    print('lista matching', lista_matching)
    rondas = get_rondas(lista_matching, n_rondas, students.index, companies.index)
    
    outputs(rondas, lista_matching)


if __name__== '__main__':
    main()



/home/alberto/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


df


,company,student,weight
0,Movetia,Maria Cristina Blanco,5.684409
1,Exoticca,Maria Cristina Blanco,4.898979
2,The Hotels Network,Maria Cristina Blanco,3.937004
3,Volcanic Internet,Maria Cristina Blanco,4.075673
4,CAPITOLE CONSULTING,Maria Cristina Blanco,5.662376
5,Farmaconnect,Maria Cristina Blanco,4.472136
6,C14TORCE,Maria Cristina Blanco,3.391165
7,AKKA,Maria Cristina Blanco,1.520437
8,Bit Genoma Digital Solutions SL,Maria Cristina Blanco,1.581139
9,Norwegian,Maria Cristina Blanco,4.500000


df normalizada


,company,student,weight
0,Movetia,Maria Cristina Blanco,0.929811
1,Exoticca,Maria Cristina Blanco,0.801337
2,The Hotels Network,Maria Cristina Blanco,0.643984
3,Volcanic Internet,Maria Cristina Blanco,0.666667
4,CAPITOLE CONSULTING,Maria Cristina Blanco,0.926207
5,Farmaconnect,Maria Cristina Blanco,0.731517
6,C14TORCE,Maria Cristina Blanco,0.554700
7,AKKA,Maria Cristina Blanco,0.248701
8,Bit Genoma Digital Solutions SL,Maria Cristina Blanco,0.258630
9,Norwegian,Maria Cristina Blanco,0.736075


df ahora cambiamos [1, 0] -> [0, 1]


,company,student,weight
0,Movetia,Maria Cristina Blanco,0.070189
1,Exoticca,Maria Cristina Blanco,0.198663
2,The Hotels Network,Maria Cristina Blanco,0.356016
3,Volcanic Internet,Maria Cristina Blanco,0.333333
4,CAPITOLE CONSULTING,Maria Cristina Blanco,0.073793
5,Farmaconnect,Maria Cristina Blanco,0.268483
6,C14TORCE,Maria Cristina Blanco,0.445300
7,AKKA,Maria Cristina Blanco,0.751299
8,Bit Genoma Digital Solutions SL,Maria Cristina Blanco,0.741370
9,Norwegian,Maria Cristina Blanco,0.263925


lista de diccionarios [{'company': 'Movetia', 'student': 'Maria Cristina Blanco', 'weight': 0.07}, {'company': 'Exoticca', 'student': 'Maria Cristina Blanco', 'weight': 0.2}, {'company': 'The Hotels Network', 'student': 'Maria Cristina Blanco', 'weight': 0.36}, {'company': 'Volcanic Internet', 'student': 'Maria Cristina Blanco', 'weight': 0.33}, {'company': 'CAPITOLE CONSULTING', 'student': 'Maria Cristina Blanco', 'weight': 0.07}, {'company': 'Farmaconnect', 'student': 'Maria Cristina Blanco', 'weight': 0.27}, {'company': 'C14TORCE', 'student': 'Maria Cristina Blanco', 'weight': 0.45}, {'company': 'AKKA', 'student': 'Maria Cristina Blanco', 'weight': 0.75}, {'company': 'Bit Genoma Digital Solutions SL', 'student': 'Maria Cristina Blanco', 'weight': 0.74}, {'company': 'Norwegian', 'student': 'Maria Cristina Blanco', 'weight': 0.26}, {'company': 'Bluekik', 'student': 'Maria Cristina Blanco', 'weight': 0.2}, {'company': 'InboundCycle', 'student': 'Maria Cristina Blanco', 'weight': 0.74},